In [1]:
import rospy
from sensor_msgs.msg import Imu

import numpy as np
import torch

In [2]:
gyro_fps=400
accel_fps=250

In [3]:
def skew2vec(W):
    assert W.shape==(3,3)
    w=torch.tensor([(W[2,1]-W[1,2])/2,
                  (W[0,2]-W[2,0])/2,
                  (W[1,0]-W[0,1])/2,
                   ])
    return w

def vec2skew(w):
    assert w.shape==(3,)
    W=torch.tensor(
                  [[0,-1*w[2],w[1]],
                  [w[2],0,-1*w[0]],
                  [-1*w[1],w[0],0]
                  ])
    return W

In [4]:
def r2w(r):
    # exponential coordinate to w,theta coordinate
    theta = torch.linalg.norm(r)
    assert 0<theta and theta<np.pi
    w = r/theta
    
    if abs(theta-0)<0.1 or abs(theta-np.pi)<0.1:
        print("warning!! theta near singularity")
    
    return theta , w

def w2r(theta,w):
    assert 0<theta and theta<np.pi
    return theta*w

In [5]:
def Rodrigues(theta,W):
    return torch.eye(3)+(torch.sin(theta)*W)+((1-torch.cos(theta))*(W@W))

def matlog(R):
    tr = torch.trace(R)
    assert abs(torch.linalg.det(R)-1)<0.0001
    assert tr<3 and tr>-1
    if abs(tr-3)<0.1 or abs(tr+1)<0.1:
        print("warning!! R near singularity")
    
    theta=torch.arccos((tr-1)/2)
    W=(R-R.T)/(2*torch.sin(theta))
    
    return theta,W

In [6]:
def aJacobian(theta,W):
    rmat=theta*W
    A=torch.eye(3) - (((1-torch.cos(theta))/(theta**2))*rmat) + (((theta-torch.sin(theta))/(theta**3))*(rmat@rmat))         
    return A

In [7]:
def dynamic_model(r,wb,dt=1/400):
    assert type(r)==torch.Tensor
    wb_=torch.tensor(wb,dtype=float)
    theta,w=r2w(r)
    W=vec2skew(w)
    A=aJacobian(theta,W)
    Ainv=torch.inverse(A)
    return r+(dt*Ainv@wb_)

In [8]:
#dynamic_model test
r=torch.tensor([0.432,1.23,0.832],dtype=float,requires_grad=True)
wb=np.array([-3.,2.,-1.])
dt=0.0025
r_next=dynamic_model(r,wb,dt)
r_next

tensor([0.4225, 1.2315, 0.8358], dtype=torch.float64, grad_fn=<AddBackward0>)

In [9]:
#ground_truth, but not precise since det|R| != 1.0000000000
theta,w=r2w(r)
W=vec2skew(w)
Wb=vec2skew(wb)
R=Rodrigues(theta,W)
Rdot=R@Wb
R_next=R+(Rdot*dt)
theta_,W_=matlog(R_next)
w_=skew2vec(W_)
r_next_=w2r(theta_,w_)
r_next_

tensor([0.4225, 1.2315, 0.8358], dtype=torch.float64, grad_fn=<MulBackward0>)

In [10]:
def measurement_model(r):
    g=torch.tensor([0,0,-9.8],dtype=float)
    
    theta,w=r2w(r)
    W=vec2skew(w)
    R=Rodrigues(theta,W)
    Rinv=torch.inverse(R)
    zhat=Rinv@g
    return zhat


In [11]:
#measurement model test
r=torch.tensor([0,np.pi/2,0],dtype=float,requires_grad=True)
ahat=measurement_model(r)
ahat

tensor([ 9.8000e+00,  0.0000e+00, -1.0880e-15], dtype=torch.float64,
       grad_fn=<MvBackward>)

In [15]:
r=torch.tensor([0.432,1.23,0.832],dtype=float,requires_grad=True)
a=np.array([0,-9.8,0])
wb=np.array([-3.,2.,-1.])
dt=0.0025


r_next=dynamic_model(r,wb,dt)
ahat=measurement_model(r)

In [16]:
r_next

tensor([0.4225, 1.2315, 0.8358], dtype=torch.float64, grad_fn=<AddBackward0>)

In [17]:
ahat

tensor([ 6.3550, -6.8281, -3.0053], dtype=torch.float64, grad_fn=<MvBackward>)

In [ ]:
class IMU_reader:
    def __init__(self):
        #accelerometer ax+b
        self.a=np.array([1.017125065,1.02456874,1.018181818])
        self.b=np.array([0.01525687597,-0.445687402,-0.3309090909])
        
        self.accel = np.zeros(3)
        self.gyro = np.zeros(3)
        rospy.init_node('listener', anonymous=True)
        rospy.Subscriber("/camera/accel/sample", Imu, self.accel_callback)
        rospy.Subscriber("/camera/gyro/sample", Imu, self.gyro_callback)

    def accel_callback(self,data):
        v=data.linear_acceleration
        v=np.array([v.x,v.y,v.z])
        self.accel=self.a*v+self.b

    def gyro_callback(self,data):
        v=data.angular_velocity
        v=np.array([v.x,v.y,v.z])
        self.gyro=v

In [ ]:
imur=IMU_reader()